In [1]:
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import popeye.og as og
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus, simulate_bar_stimulus
from popeye import css
from scipy.io import loadmat
import time
import nibabel as nib
from nilearn import plotting
import os
import tqdm
import pickle
import multiprocessing as mp
import time

# Load helper functions
from dataloader import set_paths, load_stimuli, copy_files
%load_ext autoreload
%autoreload 2

In [2]:
# Initialize parameters
params = {}
params['subjID'] = 'JC'
params['viewingDistance'] = 84 # in cm
params['screenWidth'] = 36.8 # in cm
params['scaleFactor'] = 1
params['tr_length'] = 1.3 # in seconds (got this from Aditya, need to check)
params['dtype'] = ctypes.c_int16

p = set_paths(params)

In [3]:
bar, stim_params = load_stimuli(p)
copy_files(p, params)

# Extract number of TRs
func_data = nib.load(p['pRF_func'])
params['nTRs'] = func_data.shape[-1]

Subject folder already exists


In [4]:
# create stimulus object from popeye
stimulus = VisualStimulus(bar,
                          params['viewingDistance'],
                          params['screenWidth'],
                          params['scaleFactor'],
                          params['tr_length'],
                          params['dtype'])

In [5]:
# model to fit to
method = 'ss5'
scan_data = nib.load(p['pRF_func']).get_fdata()
brainmask_data = nib.load(p['pRF_brainmask']).get_fdata() != 0
# Resample brainmask if first 2 dimensions are twice the third dimension
if brainmask_data.shape[0] == 2*brainmask_data.shape[2]:
    brainmask_data = brainmask_data[::2, ::2, :]

In [6]:
# Data to fit to
relevant_data = scan_data[brainmask_data]

# Setting up the model
css_model = css.CompressiveSpatialSummationModel(stimulus, utils.double_gamma_hrf)

css_model.hrf_delay = 0
css_model.mask_size = 1

[xi, yi, zi] = np.nonzero(brainmask_data)
indices = [(xi[i], yi[i], zi[i]) for i in range(len(xi))]

# set search grid
x_grid = (-12, 12)
y_grid = (-12, 12)
s_grid = (1/stimulus.ppd, 15)
n_grid = (0.1, 5)
grids = (x_grid, y_grid, s_grid, n_grid,)

# set search bounds
x_bounds = (-30, 30)
y_bounds = (-30, 30)
sigma_bounds = (1/css_model.stimulus.ppd, 15)
n_bounds = (0.1, 5)
beta_bounds = (1e-8, None)
baseline_bounds = (None, None)
bounds = (x_bounds, y_bounds, sigma_bounds, n_bounds, beta_bounds, baseline_bounds)

# fit settings
Ns = 10
auto_fit = 1
verbose = 2

bundle = utils.multiprocess_bundle(css.CompressiveSpatialSummationFit, css_model, relevant_data, grids, bounds, indices, auto_fit, verbose, Ns)

In [7]:
# Run the fit
start_time = time.time()
ncpu = int(os.environ.get('SLURM_CPUS_PER_TASK',default=1))
with mp.Pool(ncpu) as pool:
    results = list(tqdm.tqdm(pool.imap(utils.parallel_fit, bundle), total=len(bundle), desc='Fitting pRFs'))

end_time = time.time()

Fitting pRFs:   0%|          | 0/222873 [00:01<?, ?it/s]


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 211 and the array at index 1 has size 201